# Integrating a Small Math Dataset with VishwamAI

This notebook demonstrates how to integrate a small math dataset with VishwamAI. We will go through the following steps:
1. Loading the dataset
2. Initializing the model
3. Training the model
4. Evaluating the results


## 1. Loading the Dataset

We will use the DeepMind Mathematics dataset for this example. Let's start by loading the dataset.

In [ ]:
import datasets
from datasets import load_dataset

# Load the dataset
train_examples, val_examples = load_dataset(
    'math_dataset/arithmetic__mul',
    split=['train', 'test'],
    as_supervised=True
)

print(f"Loaded {len(train_examples)} training examples and {len(val_examples)} validation examples.")

## 2. Initializing the Model

Next, we will initialize the VishwamAI model. We will use the `train_math_model` function from `train_math.py` to set up the model.

In [ ]:
from vishwamai.architecture import init_model, VishwamaiConfig
from vishwamai.conceptual_tokenizer import ConceptualTokenizer, ConceptualTokenizerConfig
from vishwamai.training import VishwamaiTrainer
from torch.utils.data import DataLoader

# Define the model configuration
config = {
    'model_config': {
        'dim': 256,
        'n_layers': 6,
        'n_heads': 8,
        'vocab_size': 1000,
        'max_seq_len': 512
    }
}

# Initialize model configuration
model_config = VishwamaiConfig(**config['model_config'])
model = init_model(model_config)

# Initialize tokenizer
tokenizer_config = ConceptualTokenizerConfig(
    vocab_size=config['model_config'].get('vocab_size', 32000),
    max_length=config['model_config'].get('max_length', 512),
    concept_tokens=["math", "logic", "science"],
    reasoning_tokens=["if", "then", "equals", "because"]
)
tokenizer = ConceptualTokenizer(tokenizer_config)

print("Model and tokenizer initialized.")

## 3. Training the Model

We will now train the model using the `train_math_model` function from `train_math.py`.

In [ ]:
def train_math_model(train_dataset, val_dataset, config):
    # Initialize model configuration
    model_config = VishwamaiConfig(**config['model_config'])
    model = init_model(model_config)
    tokenizer_config = ConceptualTokenizerConfig(
        vocab_size=config['model_config'].get('vocab_size', 32000),
        max_length=config['model_config'].get('max_length', 512),
        concept_tokens=["math", "logic", "science"],
        reasoning_tokens=["if", "then", "equals", "because"]
    )
    tokenizer = ConceptualTokenizer(tokenizer_config)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    trainer = VishwamaiTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_loader,
        eval_dataset=val_loader,
        device="cuda"
    )
    
    trainer.train(
        num_epochs=10,
        save_dir="./models",
        evaluation_steps=100,
        save_steps=1000,
        logging_steps=10
    )

# Train the model
train_math_model(train_examples, val_examples, config)

print("Model training completed.")

## 4. Evaluating the Results

Finally, we will evaluate the trained model on the validation dataset.

In [ ]:
# Evaluate the model
trainer = VishwamaiTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_loader,
    eval_dataset=val_loader,
    device="cuda"
)

eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")